Revenue growth rate

In [23]:
# Inputs
rev_gr_1 = 12/100  # Revenue growth rate for year 1
cagr_2_5 = 12/100   # CAGR for years 2-5
risk_free_rate = 4.08/100 # Risk free rate
rev_r12m = 574785 # Revenue for the last 12 months (in millions)
ebit_r12m = 36852 # EBIT for the last 12 months (in millions)
rnd_amr_yrs = 3 # R&D amortization years
curr_rnd = 85622 # Current R&D (in millions)
prev_rnd = [73213, 56052, 42740, 0, 0, 0, 0, 0, 0, 0] # Previous R&D (in millions)
#ebit_margin_1 = 11.33/100 # Operating margin for year 1 
tgt_ebit_margin = 14/100 # Target Pre-tax Operating margin
yr_conv_margin = 5 # Years to convert to target margin
eff_tx_rate = 19/100 # Effective tax rate
mar_tax_rate = 25/100 # Marginal tax rate
sales_to_cap_1_5 = 1.5 # Sales to capital ratio Years 1-5
sales_to_cap_6_10 = 1.5 # Sales to capital ratio Years 6-10
mature_market_erp = 4.11/100 # Mature market ERP (Damodaran)
country_risk_premium = 0 # Country ERP (For US)
cost_of_capital = 8.6/100 # Cost of capital

Growth rates list

In [3]:
# Create a list of 10 growth rates
growth_rates = [rev_gr_1]  # First year growth rate

# Year two to five will be cagr_2_5
growth_rates.extend([cagr_2_5] * 4)

# Calculate the decrement for years six to ten
decrement = (cagr_2_5 - risk_free_rate) / 5

# Year six to ten growth rates
for i in range(1, 6):
    growth_rates.append(cagr_2_5 - decrement * i)

print(growth_rates)

[0.12, 0.12, 0.12, 0.12, 0.12, 0.10416, 0.08832, 0.07247999999999999, 0.056639999999999996, 0.0408]


Revenues list

In [4]:
revenues = [rev_r12m * (1 + growth_rates[0])]  # Revenue for year 1

# Calculate revenues for subsequent years
for i in range(1, len(growth_rates)):
    revenues.append(revenues[-1] * (1 + growth_rates[i]))

print(revenues) 

[643759.2000000001, 721010.3040000001, 807531.5404800002, 904435.3253376003, 1012967.5643781124, 1118478.2658837365, 1217262.266326588, 1305489.4353899392, 1379432.3570104255, 1435713.1971764509]


Amortization weights

In [5]:
# Calculate the number of non-zero entries in prev_rnd
non_zero_entries = sum(1 for rnd in prev_rnd if rnd != 0)

# Calculate the amortization weights
amortization_weights = [i / non_zero_entries for i in range(non_zero_entries, 0, -1)]

print(amortization_weights)

[1.0, 0.6666666666666666, 0.3333333333333333]


In [6]:
# Value of research asset
val_rnd = sum(a * b for a, b in zip(([curr_rnd] + prev_rnd)[:len(amortization_weights)], amortization_weights))
val_rnd

153114.66666666666

In [7]:
#Amortization for this year
sum_prev_rnd = sum([rnd / rnd_amr_yrs for rnd in prev_rnd])
sum_prev_rnd

57335.0

In [8]:
# Adjustment to EBIT due to R&D
adj_ebit = curr_rnd - sum_prev_rnd
adj_ebit

28287.0

Adjusted operating income

In [9]:
# Adjusted EBIT (Base Year)
if curr_rnd != 0:
    ebit_adj = ebit_r12m + adj_ebit
else: 
    ebit_adj = ebit_r12m
ebit_adj

65139.0

Operating Margins

In [10]:
ebit_margin_0 = ebit_adj / rev_r12m
ebit_margin_1 = ebit_margin_0 # Assuming same margin for year 1

In [11]:
# Initialize the list with the operating margin for the first year
operating_margins = [ebit_margin_1]

# Calculate the step change for the operating margin
step_change = (tgt_ebit_margin - ebit_margin_1) / (yr_conv_margin)

# Calculate the operating margins for years 2 to the year of convergence
for i in range(1, yr_conv_margin):
    operating_margins.append(tgt_ebit_margin - step_change*(yr_conv_margin - i - 1))

# From the year of convergence onwards, the operating margin remains the same as the target one
operating_margins.extend([tgt_ebit_margin] * (10 - yr_conv_margin + 1))

print(operating_margins)

[0.11332759205616012, 0.12399655523369607, 0.12933103682246405, 0.13466551841123203, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14]


Operating Income

In [12]:
ebits = [revenue * margin for revenue, margin in zip(revenues, operating_margins)]
print(ebits)

[72955.68000000001, 89402.793984, 104438.89139712002, 121796.25195601924, 141815.45901293575, 156586.95722372312, 170416.71728572235, 182768.5209545915, 193120.5299814596, 200999.84760470313]


In [13]:
# Initialize the list with the effective tax rate for the first 5 years
tax_rates = [eff_tx_rate] * 5

# Calculate the step change for the tax rate
tax_rate_step_change = (mar_tax_rate - eff_tx_rate) / 5

# Calculate the tax rates for years 6 to 10
for i in range(1, 6):
    tax_rates.append(eff_tx_rate + tax_rate_step_change * i)

print(tax_rates)

[0.19, 0.19, 0.19, 0.19, 0.19, 0.202, 0.214, 0.226, 0.238, 0.25]


EBIT(1-tax)

In [14]:
ebit_minus_tax = [ebit * (1 - tax_rate) for ebit, tax_rate in zip(ebits, tax_rates)]
print(ebit_minus_tax)

[59094.10080000001, 72416.26312704, 84595.50203166723, 98654.96408437559, 114870.52180047797, 124956.39186453106, 133947.53978657778, 141462.83521885384, 147157.8438458722, 150749.88570352734]


Reinvestment

In [16]:
# Initialize the list of reinvestment
reinvestment = []

# Calculate reinvestment for years 1 to 5 using sales_to_cap_1_5
for i in range(5):
    reinvestment.append((revenues[i + 1] - revenues[i]) / sales_to_cap_1_5)

# Calculate reinvestment for years 6 to 9 using sales_to_cap_6_10
for i in range(5, 9):
    reinvestment.append((revenues[i + 1] - revenues[i]) / sales_to_cap_6_10)

# Calculate terminal year revenue
terminal_year_revenue = revenues[9] * (1 + risk_free_rate)

# Calculate reinvestment for year 10
reinvestment.append((terminal_year_revenue - revenues[9]) / sales_to_cap_6_10)

print(reinvestment)

[51500.73600000003, 57680.82432000005, 64602.52323840008, 72354.82602700808, 70340.46767041607, 65856.00029523438, 58818.11270890074, 49295.2810803242, 37520.56011068355, 39051.39896319946]


FCFF

In [19]:
fcff = [ebit - reinvest for ebit, reinvest in zip(ebit_minus_tax, reinvestment)]
print(fcff)

[7593.364799999974, 14735.438807039951, 19992.978793267153, 26300.138057367512, 44530.05413006191, 59100.391569296684, 75129.42707767704, 92167.55413852964, 109637.28373518866, 111698.48674032788]


Terminal cost of capital

In [22]:
terminal_cost_of_capital = risk_free_rate + mature_market_erp + country_risk_premium

In [24]:
# Initialize the list with the cost of capital for the first 5 years
cost_of_capital_list = [cost_of_capital] * 5

# Calculate the step change for the cost of capital
cost_of_capital_step_change = (terminal_cost_of_capital - cost_of_capital) / 5

# Calculate the cost of capital for years 6 to 10
for i in range(1, 6):
    cost_of_capital_list.append(cost_of_capital + cost_of_capital_step_change * i)

print(cost_of_capital_list)

[0.086, 0.086, 0.086, 0.086, 0.086, 0.08517999999999999, 0.08435999999999999, 0.08354, 0.08272, 0.0819]


In [25]:
# Initialize the list of discount factors with the first year's discount factor
discount_factors = [1 / (1 + cost_of_capital_list[0])]

# Calculate the discount factors for subsequent years
for i in range(1, 10):
    discount_factors.append(discount_factors[-1] / (1 + cost_of_capital_list[i]))

print(discount_factors)

[0.9208103130755064, 0.8478916326662121, 0.7807473597294771, 0.7189202207453748, 0.6619891535408607, 0.6100270494672412, 0.562568749739239, 0.5191951840626456, 0.4795285799307722, 0.44322819108122025]


Present value of FCFF

In [26]:
pv_fcff = [fcf * discount_factor for fcf, discount_factor in zip(fcff, discount_factors)]
print(pv_fcff)

[6992.048618784506, 12494.055268154165, 15609.465405970757, 18907.701057836483, 29478.412840688394, 36052.837491376675, 42265.4678597141, 47852.95023555775, 52574.21097700216, 49507.91822442519]
